In [1]:
%pip install transformers

Looking in indexes: https://nexus.iisys.de/repository/ki-awz-pypi-group/simple, https://pypi.org/simple
Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install 'accelerate>=0.26.0'

Looking in indexes: https://nexus.iisys.de/repository/ki-awz-pypi-group/simple, https://pypi.org/simple
Note: you may need to restart the kernel to use updated packages.


In [3]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

2024-11-26 15:23:50.726063: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-26 15:23:50.764647: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-128k-instruct",
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=True,
)

`flash-attention` package not found, consider installing for better performance: No module named 'flash_attn'.
Current `flash-attention` does not support `window_size`. Either upgrade or use `attn_implementation='eager'`.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

### number of max new tokens = 50. for 1 sentence test

In [5]:
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-128k-instruct")
pipe = pipeline('text-generation', model=model, tokenizer=tokenizer,)

generation_args = {
    "max_new_tokens": 50,
    "return_full_text": False,
    "temperature": 0.7,
    "do_sample": True,
}

In [7]:
def get_color_from_description(description):
    prompt = f"""
    You are an assistant that generates colors in RGB format based on the mood described in a given sentence. 
    Your task is to think about the emotional tone or atmosphere of the sentence and generate a color that best represents it. 
    The color should be in the RGB format with three values, where R, G, and B are numbers between 0 and 255. 
    
    Example 1: 
    - Sentence: "A sunny beach day with clear skies."
    - Output: "255, 223, 0"  (This represents a bright yellow, which conveys joy and sunshine.)
    
    Example 2: 
    - Sentence: "A calm and peaceful sunset by the sea."
    - Output: "255, 94, 77"  (This represents a soft romantic pinkish color.)
    
    Now, please generate a color in RGB format for the following description:
    "{description}"
    """
    response = pipe(prompt, **generation_args)
    generated_text = response[0]['generated_text'].strip()
    rgb_color = re.search(r'(\d{1,3}),\s*(\d{1,3}),\s*(\d{1,3})', generated_text)
    if rgb_color:
        return tuple(map(int, rgb_color.groups()))
    else:
        return "255, 255, 255"

In [ ]:
import re
import json
descriptions = [
    "Sunny day at the beach.",
    "Romantic evening with a sunset.",
    "A stormy night with dark clouds.",
    "Black"
]

color_data = []

for description in descriptions:
    rgb_color = get_color_from_description(description)
    color_data.append({
            "description": description,
        "color": rgb_color}
    )
    print(rgb_color)

with open("color_description.json", "w") as json_file:
    json.dump(color_data, json_file, indent=4)

(255, 223, 0)
(255, 210, 220)
(0, 51, 102)
